# 2. FERPlus CNN01+SVM+XGBoost 분류

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost as xgb
import tensorflow as tf
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

import keras.backend.tensorflow_backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, concatenate, Input, Conv2D, MaxPooling2D
from keras.optimizers import Adam, Adadelta
from keras.layers.advanced_activations import LeakyReLU
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## 2.1. 학습에 사용될 하이퍼 패러미터 선언 및 Random Seed 설정

In [2]:
np.random.seed(201)

In [3]:
FERPLUS_DIR = "datasets/ferPlus/prep/"
INPUT_TRAIN_DATA = 'ferPlus_X.npy'
LABEL_TRAIN_DATA = 'ferPlus_y.npy'

TEST_SPLIT = 0.2
RND_SEED = 0

In [4]:
input_data = np.load(open(FERPLUS_DIR + INPUT_TRAIN_DATA, 'rb'))
label_data = np.load(open(FERPLUS_DIR + LABEL_TRAIN_DATA, 'rb'))

In [5]:
input_shape = input_data[1].shape
print('Input Shape is :', input_shape)

Input Shape is : (48, 48, 1)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(input_data,
                                                    label_data,
                                                    test_size=TEST_SPLIT,
                                                    random_state=RND_SEED)

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train,
                                                  test_size=TEST_SPLIT,
                                                  random_state=RND_SEED)

In [9]:
x_train.shape

(8784, 48, 48, 1)

In [10]:
x_val.shape

(2196, 48, 48, 1)

In [11]:
x_test.shape

(2746, 48, 48, 1)

## 2.2 CNN모델 정의

In [ ]:
def fire_incept(x, fire=16, intercept=64):
    x = Conv2D(fire, (5,5), strides=(2,2))(x)
    x = LeakyReLU(alpha=0.15)(x)
    
    left = Conv2D(intercept, (3,3), padding='same')(x)
    left = LeakyReLU(alpha=0.15)(left)
    
    right = Conv2D(intercept, (5,5), padding='same')(x)
    right = LeakyReLU(alpha=0.15)(right)
    
    x = concatenate([left, right], axis=3)
    return x

In [ ]:
def fire_squeeze(x, fire=16, intercept=64):
    x = Conv2D(fire, (1,1))(x)
    x = LeakyReLU(alpha=0.15)(x)
    
    left = Conv2D(intercept, (1,1))(x)
    left = LeakyReLU(alpha=0.15)(left)
    
    right = Conv2D(intercept, (3,3), padding='same')(x)
    right = LeakyReLU(alpha=0.15)(right)
    
    x = concatenate([left, right], axis=3)
    return x

In [ ]:
image_input=Input(shape=input_shape)

x = fire_incept((image_input), fire=16, intercept=16)

x = fire_incept(x, fire=32, intercept=32)
x = fire_squeeze(x, fire=32, intercept=32)

x = fire_incept(x, fire=64, intercept=64)
x = fire_squeeze(x, fire=64, intercept=64)

x = Conv2D(64, (3,3))(x)
x = LeakyReLU(alpha=0.1)(x)

x = Flatten()(x)

x = Dense(512)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dropout(0.1)(x)

out = Dense(2, activation='softmax')(x)

모델 컴파일

In [ ]:
model = Model(image_input, out)
model.summary()

모델 플롯 생성

In [ ]:
tf.keras.utils.plot_model(model, to_file='cnn/plot/ferPlus/model.png')

In [ ]:
model.compile(optimizer = Adam(lr=.00025) , loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
MODEL_PATH = 'cnn/weight/ferPlus/model_weights.h5f'

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
check_point = ModelCheckpoint(filepath=MODEL_PATH, monitor='val_accuracy', verbose=1, save_best_only=True)

## 2.3 CNN 모델 훈련

In [ ]:
batch_size = 32
epochs = 40

In [ ]:
# Training
with K.tf_ops.device('/device:GPU:0'):
    history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size,
                        validation_data = (x_val, y_val), callbacks=[learning_rate_reduction, check_point])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training for ' +str(epochs)+ ' epochs')
plt.legend(['Training accuracy', 'Validation accuracy'], loc='lower right')
plt.show()

검증셋 정확도

In [ ]:
model.load_weights(MODEL_PATH)
results = model.evaluate(x_val, y_val)
print('Test accuracy: ', results[1])

## 2.4 SVM 적용

마지막 레이어에서 특징추출

In [ ]:
model_feat = Model(inputs=model.input,outputs=model.get_layer('dense_1').output)

feat_train = model_feat.predict(x_train)
print(feat_train.shape)

feat_val = model_feat.predict(x_val)
print(feat_val.shape)

feat_test = model_feat.predict(x_test)
print(feat_test.shape)

SVM에 적용

In [ ]:
svm = SVC(kernel='rbf')

svm.fit(feat_train, np.argmax(y_train,axis=1))

print('fitting done !!!')

훈련셋 정확도

In [ ]:
svm.score(feat_train, np.argmax(y_train,axis=1))

검증셋 정확도

In [ ]:
svm.score(feat_val, np.argmax(y_val,axis=1))

SVM의 Prediction 저장

In [ ]:
prediction = svm.predict(feat_test)

## 2.5 XGBoost 적용

In [ ]:
xb = xgb.XGBClassifier()

xb.fit(feat_train, np.argmax(y_train,axis=1))
print('fitting done !!!')

검증셋 정확도

In [ ]:
xb.score(feat_val, np.argmax(y_val,axis=1))

XGBoost의 Prediction 저장

In [ ]:
prediction = xb.predict(feat_test)

최종 정확도 측정

In [ ]:
xb.score(feat_test, np.argmax(y_test,axis=1))